In [1]:
import os
import uuid
import pandas as pd
import numpy as np
from predictor import MultiModalPredictor

# Create a sample dataset with images, masks, and point prompts
def create_sample_dataset(num_samples=10):
    """
    Create a sample dataset for semantic segmentation with point prompts.
    
    This is just a placeholder - in a real scenario, you would use your actual data.
    """
    data = {
        'image': [f'path/to/image_{i}.jpg' for i in range(num_samples)],
        'label': [f'path/to/mask_{i}.png' for i in range(num_samples)],
        'points': []
    }
    
    # Create sample point prompts
    # Format: "x1,y1,label1;x2,y2,label2;..." where coordinates are normalized [0-1]
    for i in range(num_samples):
        num_points = np.random.randint(1, 5)  # 1-4 points per image
        points = []
        
        for j in range(num_points):
            x = np.random.random()  # Normalized x coordinate [0-1]
            y = np.random.random()  # Normalized y coordinate [0-1]
            label = 1  # Foreground point (1 = foreground, 0 = background)
            points.append(f"{x:.4f},{y:.4f},{label}")
        
        data['points'].append(';'.join(points))
    
    return pd.DataFrame(data)

# Create sample datasets
train_data = create_sample_dataset(20)
val_data = create_sample_dataset(5)

# Print sample data
print("Sample training data:")
print(train_data.head(2))

# Create a unique save path
save_path = f"./tmp/{uuid.uuid4().hex}-sam_with_points"
os.makedirs(save_path, exist_ok=True)

# Initialize the predictor with SAM and LoRA fine-tuning
predictor = MultiModalPredictor(
    problem_type="semantic_segmentation", 
    label="label",
    hyperparameters={
        "model.sam.checkpoint_name": "facebook/sam-vit-base",
        "optimization.efficient_finetune": "lora",
        "model.sam.max_points_per_image": 10,  # Maximum number of points to use per image
        "model.sam.point_missing_strategy": "empty",  # How to handle missing points
    },
    path=save_path,
)

# Train the model
predictor.fit(
    train_data=train_data,
    tuning_data=val_data,
    time_limit=1800,  # 30 minutes
)

# Make predictions
predictions = predictor.predict(val_data)
print(f"Predictions saved to: {predictions}")

# You can also evaluate the model
evaluation = predictor.evaluate(val_data)
print(f"Evaluation results: {evaluation}") 

ModuleNotFoundError: No module named 'transformers'